In [151]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os  

In [152]:
titanic_df = pd.read_csv("./data/train.csv")
titanic_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [153]:
titanic_df["sex_enc"] = np.where(titanic_df["Sex"]=="male", 1, 0)

In [154]:
mean_age = titanic_df["Age"].mean()
titanic_df["new_age"]= titanic_df["Age"].fillna(mean_age)

In [155]:
age_bins = [0, 20, 40, 60, 80, 100]
age_lbl = [20, 40, 60, 80, 100]
titanic_df['age_bin'] = pd.cut(titanic_df['new_age'], bins= age_bins, labels=age_lbl)
titanic_df = pd.get_dummies(titanic_df, columns=["age_bin"])

In [156]:
titanic_df = pd.get_dummies(titanic_df, columns=["Embarked"])
titanic_df["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [157]:
titanic_df = pd.get_dummies(titanic_df, columns=["Pclass"])

In [158]:
train, val, test = np.split(titanic_df.sample(frac=1), [int(0.8*len(titanic_df)), int(0.9*len(titanic_df))])
feature_list = ["sex_enc", "Parch", "SibSp", "Embarked_C", "Embarked_Q", "Embarked_S"
                , "Pclass_1", "Pclass_2", "Pclass_3", "age_bin_20", "age_bin_40", "age_bin_60", "age_bin_80", "age_bin_100"]

In [159]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Survived')
  ds = tf.data.Dataset.from_tensor_slices((dataframe[feature_list], labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [160]:
batch_size = 10
train_ds = df_to_dataset(train, shuffle=False, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [161]:
input_shape = (len(feature_list),)
inputs = tf.keras.layers.Input(shape=input_shape)
x = tf.keras.layers.Dense(32, activation="relu")(inputs)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs, output)
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
72/72 [==============================] - 0s 2ms/step - loss: 0.7103 - accuracy: 0.4986 - val_loss: 0.6279 - val_accuracy: 0.6404
Epoch 2/50
72/72 [==============================] - 0s 973us/step - loss: 0.6461 - accuracy: 0.6236 - val_loss: 0.5817 - val_accuracy: 0.7079
Epoch 3/50
72/72 [==============================] - 0s 930us/step - loss: 0.6125 - accuracy: 0.6770 - val_loss: 0.5551 - val_accuracy: 0.7416
Epoch 4/50
72/72 [==============================] - 0s 1ms/step - loss: 0.5944 - accuracy: 0.7149 - val_loss: 0.5347 - val_accuracy: 0.7416
Epoch 5/50
72/72 [==============================] - 0s 1ms/step - loss: 0.5752 - accuracy: 0.7233 - val_loss: 0.5188 - val_accuracy: 0.7753
Epoch 6/50
72/72 [==============================] - 0s 1ms/step - loss: 0.5673 - accuracy: 0.7233 - val_loss: 0.5047 - val_accuracy: 0.7753
Epoch 7/50
72/72 [==============================] - 0s 997us/step - loss: 0.5678 - accuracy: 0.7444 - val_loss: 0.4922 - val_accuracy: 0.7978
Epoch 8/50
72/

In [162]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

9/9 [==============================] - 0s 877us/step - loss: 0.3691 - accuracy: 0.8333
Loss:  0.36911922693252563
Accuracy:  0.8333333134651184


In [164]:
test_data = pd.read_csv("./data/test.csv")
pred_in = test_data.copy()
pred_in["sex_enc"] = np.where(pred_in["Sex"]=="male", 1, 0)
pred_in["new_age"] = pred_in["Age"].fillna(mean_age)
pred_in = pd.get_dummies(pred_in, columns=["Embarked"])
pred_in = pd.get_dummies(pred_in, columns=["Pclass"])
pred_in['age_bin'] = pd.cut(pred_in['new_age'], bins= age_bins, labels=age_lbl)
pred_in = pd.get_dummies(pred_in, columns=["age_bin"])

In [165]:
pred_ds = tf.data.Dataset.from_tensor_slices(pred_in[feature_list])
pred_ds = pred_ds.batch(batch_size)

In [166]:
predictions = model.predict(pred_ds)
pred_pd = pd.DataFrame(predictions, columns=["pred_survived"])

In [171]:
pred_pd["Survived"] = np.where(pred_pd["pred_survived"]>0.5, 1, 0)

In [172]:
out = pd.merge(pred_in, pred_pd, left_index=True, right_index=True)

In [173]:
out_pd = out[["PassengerId","Survived"]]

In [174]:
os.makedirs('./data/', exist_ok=True)  
out_pd.to_csv('./data/out.csv',index=False)  